


# <center>  Regresión Lineal y Regularización </center>

## Descripción
Esta sección conlleva la aplicación de los algoritmos: Regresión Lineal, Ridge y LASSO

## Contenido
* Importación de librerias y módulos
* Carga dataset de trabajo
* Análisis exploratorio de datos
* Desarrollo del modelo

## Requisitos previos

* Haber completado los cursos:
  - Introducción a Python
  - Estadística para Ciencia de Datos



### Importar módulos y verificar versiones

In [0]:
import sys
import numpy as np
import pandas as pd
import sklearn as sk
import matplotlib
import seaborn as sns

print('Python:', sys.version)
print('NumPy:', np.__version__)
print('Pandas:', pd.__version__)
print('Seaborn:', sns.__version__)
print('Matplotlib:', matplotlib.__version__)
print('Scikit-learn:', sk.__version__)

## 1. Importar módulos específicos de librerías

In [0]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
import matplotlib.pyplot as plt

## 2. Dataset de Trabajo

**restaurantes:** Información de las evaluaciones realizadas a los restaurantes italianos por parte de los comensales en la ciudad de NY (revista ZAGAT 2013).

<a href='https://storage.googleapis.com/datasets-academy/Track%20Data%20Science/02%20-%20Introduccion%20a%20Machine%20Learning/NY_italian_restaurants.csv'>
  Link para descargar el dataset NY_italian_restaurants</a>

**Objetivo de Regresión:** Predecir los precios medios de la comida de los restaurantes a través de las evaluaciones en los clientes sobre la comida, decoración y servicio.

**Diccionario de datos**
* Case: Número del restaurante
* Restaurant: Nombre del restaurante
* Price: Precio medio de la comida (incluye bebida y propina)
* Food: Puntaje de 0 a 30 por la calidad en la comida
* Decor: Puntaje de 0 a 30 por la calidad en la decoración
* Service: Puntaje de 0 a 30 por la calidad en el servicio
* East: Igual a 1 si se encuentra al Este de la 5ta avenida y 0 si al Oeste

In [0]:
# Cargar el dataset
restaurantes = pd.read_csv('https://storage.googleapis.com/datasets-academy/Track%20Data%20Science/02%20-%20Introduccion%20a%20Machine%20Learning/NY_italian_restaurants.csv')
restaurantes.head()

In [0]:
restaurantes.shape

### 2.1 Análisis exploratorio de datos

In [0]:
restaurantes.dtypes

In [0]:
restaurantes.describe(include = np.number)

Analísis de las correlaciones entre las variables

In [0]:
restaurantes.iloc[:, 1:6].corr()

Análisis visual de las funciones de densidad por variable y las relaciones entre variables mediante gráficos de dispersión

In [0]:
sns.pairplot(restaurantes, 
             hue='East',
             x_vars=['Price', 'Food', 'Decor', 'Service'],
             y_vars=['Price', 'Food', 'Decor', 'Service'])

## 3. Desarrollo del Modelo

### 3.1 Preparación de los datos correspondientes a las variables ```y , X```

Igualmente, es recomendable que la selección de los datos correspondientes a ```y, X``` finalice en arreglos numpy para facilidad de manipulación en los algoritmos de ```sklearn```.<br>

In [0]:
# Preparar los datos
y = restaurantes['Price'].values
X = restaurantes[['Food','Decor','Service']].values

### 3.2 División de los datos: Training y Test

**```random_state```** <br>
- Este parámetro inicializa el generador aleatorio de números que indicará los índices de división los datos en las porciones de Training y Test. 
- Para obtener las mismas divisiones de datos en diferentes computadoras, fijamos este parámetro en un mismo valor, conocido como valor semilla. En nuestros ejercicios utilizaremos la semilla 0.

In [0]:
# Separamos en training y test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=0)

In [0]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

### 3.3 Creación de los algoritmos

Vamos a realizar una comparación de los Modelos de Regresión Lineal, Ridge y LASSO

In [0]:
#Creamos el algoritmo de Regresión Lineal
lineal = LinearRegression()
lineal

In [0]:
# Creación del algoritmo Ridge
ridge = Ridge()
ridge

In [0]:
# Creación del algoritmo LASSO
lasso = Lasso()
lasso

### 3.4 Validación cruzada

Aplicamos validación para encontrar el valor de alpha (penalización lambda)

In [0]:
#Definimos el número de k folds (bloques) y los hiperparámetros
k = 10
grid_values = {'alpha': [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000]}

In [0]:
#Aplicamos la validación cruzada
grid_ridge = GridSearchCV(estimator=ridge, 
                          param_grid=grid_values, 
                          cv=k, 
                          scoring='neg_root_mean_squared_error', 
                          return_train_score=False, 
                          n_jobs = -1)

grid_ridge.fit(X = X_train, y = y_train)

#Resultados GridSearchCV
df_cv = pd.DataFrame(grid_ridge.cv_results_)
df_cv

In [0]:
# Imprimimos los atributos de respuesta de CV para Ridge
print('Mejor métrica (promedio en test):', grid_ridge.best_score_*-1)
print('Hiperparámetro óptimo:', grid_ridge.best_params_)

In [0]:
#Aplicamos la validación cruzada
grid_lasso = GridSearchCV(estimator=lasso, 
                          param_grid=grid_values, 
                          cv=k, 
                          scoring='neg_root_mean_squared_error', 
                          return_train_score=False, 
                          n_jobs=-1)

grid_lasso.fit(X = X_train, y = y_train)

#Resultados GridSearchCV
df_cv = pd.DataFrame(grid_lasso.cv_results_)
df_cv

In [0]:
# Imprimimos los atributos de respuesta de CV para LASSO
print('Mejor métrica (promedio en test):', grid_lasso.best_score_*-1)
print('Hiperparámetro óptimo:', grid_lasso.best_params_)

### 3.5 Prediccion a partir del algoritmo entrenado

In [0]:
#Ajustamos el algoritmo con los datos de training
lineal.fit(X_train, y_train)

In [0]:
#Analizamos los coeficientes para la Regresión Lineal
print(pd.Series(lineal.coef_.tolist(), index = restaurantes.columns[3:6]))

In [0]:
#Ajustamos el algoritmo con su hiperparámentro óptimo con los datos de training
ridge_opt = Ridge(alpha = grid_ridge.best_params_['alpha'])
ridge_opt.fit(X_train, y_train)

In [0]:
#Analizamos los coeficientes de Ridge
print(pd.Series(ridge_opt.coef_.tolist(), index = restaurantes.columns[3:6]))

In [0]:
#Ajustamos el algoritmo con su hiperparámentro óptimo con los datos de training
lasso_opt = Lasso(grid_lasso.best_params_['alpha'])
lasso_opt.fit(X_train, y_train)

In [0]:
#Analizamos los coeficientes de LASSO
print(pd.Series(lasso_opt.coef_.tolist(), index = restaurantes.columns[3:6]))

In [0]:
# Predecimos con nuestro algoritmo en los datos training (analizar overfitting)
yHat_train_lineal = lineal.predict(X_train)
yHat_train_ridge = ridge_opt.predict(X_train)
yHat_train_lasso = lasso_opt.predict(X_train)

In [0]:
# Predecimos con nuestro algoritmo en los datos test
yHat_test_lineal = lineal.predict(X_test)
yHat_test_ridge = ridge_opt.predict(X_test)
yHat_test_lasso = lasso_opt.predict(X_test)

### 3.6 Evaluación del algoritmo

Evaluación en datos de training (analizar overfitting al comparar con la evaluación en test)

In [0]:
print('Resultados Regresión Lineal:')
print('Error Cuadrático Medio (MSE):', metrics.mean_squared_error(y_train, yHat_train_lineal))
print('La raíz del Error Cuadrático Medio (RMSE):', metrics.mean_squared_error(y_train, yHat_train_lineal, squared=False))
print('Coeficiente de determinación R^2:', metrics.r2_score(y_train, yHat_train_lineal))

In [0]:
print('Resultados Regresión Ridge:')
print('Error Cuadrático Medio (MSE):', metrics.mean_squared_error(y_train, yHat_train_ridge))
print('La raíz del Error Cuadrático Medio (RMSE):', metrics.mean_squared_error(y_train, yHat_train_ridge, squared=False))
print('Coeficiente de determinación R^2:', metrics.r2_score(y_train, yHat_train_ridge))

In [0]:
print('Resultados Regresión LASSO:')
print('Error Cuadrático Medio (MSE):', metrics.mean_squared_error(y_train, yHat_train_lasso))
print('La raíz del Error Cuadrático Medio (RMSE):', metrics.mean_squared_error(y_train, yHat_train_lasso, squared=False))
print('Coeficiente de determinación R^2:', metrics.r2_score(y_train, yHat_train_lasso))

Evaluación en datos de test

In [0]:
print('Resultados Regresión Lineal:')
print('Error Cuadrático Medio (MSE):', metrics.mean_squared_error(y_test, yHat_test_lineal))
print('La raíz del Error Cuadrático Medio (RMSE):', metrics.mean_squared_error(y_test, yHat_test_lineal, squared=False))
print('Coeficiente de determinación R^2:', metrics.r2_score(y_test, yHat_test_lineal))

In [0]:
print('Resultados Regresión Ridge:')
print('Error Cuadrático Medio (MSE):', metrics.mean_squared_error(y_test, yHat_test_ridge))
print('La raíz del Error Cuadrático Medio (RMSE):', metrics.mean_squared_error(y_test, yHat_test_ridge, squared=False))
print('Coeficiente de determinación R^2:', metrics.r2_score(y_test, yHat_test_ridge))

In [0]:
print('Resultados Regresión Lasso:')
print('Error Cuadrático Medio (MSE):', metrics.mean_squared_error(y_test, yHat_test_lasso))
print('La raíz del Error Cuadrático Medio (RMSE):', metrics.mean_squared_error(y_test, yHat_test_lasso, squared=False))
print('Coeficiente de determinación R^2:', metrics.r2_score(y_test, yHat_test_lasso))

%md > **Ejercicio:**

1. Incluir como predictor a la ubicación del restaurante (variable East) en la matriz `X_nueva`
2. Replicar el flujo del proceso de Machine Learning:
    - División de datos: training 80% y test 20%
    - Creación del algortimo LASSO()
    - Ejecutar validación cruzada, con el siguiente grid de valores `'alpha': [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000]`
    - Ajuste del algoritmo optimizado
    - Predicción con los datos de test
    - Obtención de métricas de evaluación (MSE, RMSE y R2)
3. Comparar la métrica del RMSE del primer modelo LASSO (que no incluye East), con el segundo modelo LASSO que incluye el predictor East. ¿Cúal modelo tiene un mejor desempeño al evaluar las predicciones con los datos de test?

In [0]:
# Su código aquí


In [0]:
#Selección de los predictores
y = restaurantes['Price'].values
X_nueva = restaurantes[['Food','Decor','Service', 'East']].values

#Division training - test
X_train, X_test, y_train, y_test = train_test_split(X_nueva, y, test_size = 0.2, random_state=0)

#Creación del algoritmo LASSO
lasso = Lasso()
lasso

#Definimos el número de k folds (bloques) y los hiperparámetros
k = 10
grid_values = {'alpha': [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000]}

#Aplicamos la validación cruzada
grid_lasso = GridSearchCV(estimator=lasso, 
                          param_grid=grid_values, 
                          cv=k, 
                          scoring='neg_root_mean_squared_error', 
                          return_train_score=False, 
                          n_jobs=-1)

grid_lasso.fit(X = X_train, y = y_train)

#Ajustamos el algoritmo con su hiperparámentro óptimo con los datos de training
lasso_opt = Lasso(grid_lasso.best_params_['alpha'])
lasso_opt.fit(X_train, y_train)

#Predecimos con nuestro algoritmo en los datos test
yHat_test_lasso = lasso_opt.predict(X_test)

#Obtención de las métricas de evaluación
print('Resultados Regresión Lasso en TEST:')
print('Error Cuadrático Medio (MSE):', metrics.mean_squared_error(y_test, yHat_test_lasso))
print('La raíz del Error Cuadrático Medio (RMSE):', metrics.mean_squared_error(y_test, yHat_test_lasso, squared=False))
print('Coeficiente de determinación R^2:', metrics.r2_score(y_test, yHat_test_lasso))